<a href="https://colab.research.google.com/github/kito2007/CV_assignment/blob/main/CV_%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

# 데이터 경로 설정
data_dir = '/content/drive/MyDrive/CV/chest_xray'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
val_dir = os.path.join(data_dir, 'val')
labels = ['NORMAL', 'PNEUMONIA']
img_size = 224  # VGG16 모델의 입력 이미지 크기

# 데이터 불러오기 및 전처리 함수
def load_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        files = os.listdir(path)
        np.random.shuffle(files)
        count = 0
        for img_file in files:
            if count >= 500:  # 각 클래스별로 500개씩 사용
                break
            if img_file.endswith('.DS_Store'):  # .DS_Store 파일 제외
                continue
            img_path = os.path.join(path, img_file)
            try:
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (img_size, img_size))
                data.append([img, class_num])
                count += 1
            except Exception as e:
                print(f"이미지 경로 예외처리: {img_path}")
                print(e)
    return np.array(data)



# 데이터 로드
train_data = load_data(train_dir)
test_data = load_data(test_dir)
val_data = load_data(val_dir)

# 데이터셋 분할
train_images = np.array([item[0] for item in train_data])
train_labels = np.array([item[1] for item in train_data])
test_images = np.array([item[0] for item in test_data])
test_labels = np.array([item[1] for item in test_data])
val_images = np.array([item[0] for item in val_data])
val_labels = np.array([item[1] for item in val_data])

# 전처리 및 데이터 증강
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=32)

# VGG16 모델 로드
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# 모델 구성
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

# 모델 학습
model.fit(train_generator, epochs=10, validation_data=test_generator)

# 모델 평가
loss, accuracy = model.evaluate(test_generator)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


<ipython-input-7-e391d5383acd>:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


Epoch 1/10
32/32 [==============================] - 16s 451ms/step - loss: 0.5082 - accuracy: 0.7410 - val_loss: 0.4370 - val_accuracy: 0.8285
Epoch 2/10
32/32 [==============================] - 14s 444ms/step - loss: 0.2479 - accuracy: 0.9030 - val_loss: 0.5366 - val_accuracy: 0.7901
Epoch 3/10
32/32 [==============================] - 14s 448ms/step - loss: 0.1991 - accuracy: 0.9210 - val_loss: 0.4048 - val_accuracy: 0.8494
Epoch 4/10
32/32 [==============================] - 14s 450ms/step - loss: 0.1651 - accuracy: 0.9380 - val_loss: 0.4062 - val_accuracy: 0.8189
Epoch 5/10
32/32 [==============================] - 15s 454ms/step - loss: 0.1068 - accuracy: 0.9600 - val_loss: 0.6024 - val_accuracy: 0.8077
Epoch 6/10
32/32 [==============================] - 15s 455ms/step - loss: 0.1157 - accuracy: 0.9520 - val_loss: 0.4542 - val_accuracy: 0.8253
Epoch 7/10
32/32 [==============================] - 15s 458ms/step - loss: 0.0889 - accuracy: 0.9650 - val_loss: 0.7591 - val_accuracy: 0.7788

In [ ]:
from tensorflow.keras.models import load_model

# 예측 함수 정의
def predict_images(model, data_dir):
    data = load_data(data_dir)
    images = np.array([item[0] for item in data])
    labels = np.array([item[1] for item in data])

    # 예측 수행
    predictions = model.predict(images)
    predicted_labels = np.argmax(predictions, axis=1)

    # 예측 결과 출력
    for i in range(len(images)):
        print("실제 레이블:", labels[i])
        print("예측 레이블:", predicted_labels[i])
        print()

# 모델 로드
model = load_model('model.h5')

# 예측 수행 및 결과 출력
predict_images(model, '/content/drive/MyDrive/CV/chest_xray/val')


OSError: ignored